In [ ]:
import json
import uproot
import awkward as ak
import vector
import numpy as np
import matplotlib.pyplot as plt
import warnings

# plotting params
plt.rcParams.update({
    'figure.figsize': (10, 6),
    'axes.grid': True,
    'grid.alpha': 0.6,
    'grid.linestyle': '--',
    'font.size': 14,
    "figure.dpi": 200,
})

# Suppress a harmless warning from the vector library with awkward arrays
warnings.filterwarnings("ignore", message="Passing an awkward array to a ufunc")

# Register the vector library with awkward array
ak.behavior.update(vector.backends.awkward.behavior)

# --- 1. CONFIGURATION ---
# All user-changable settings are here.
with open("hh-bbbb-obj-config.json", "r") as config_file:
    CONFIG = json.load(config_file)

# --- 2. DATA LOADING & PREPARATION FUNCTIONS ---
from data_loading_helpers import load_and_prepare_data, select_gen_b_quarks_from_higgs

# --- 3. ANALYSIS FUNCTIONS ---
from analysis_helpers import *

# --- 4. PLOTTING FUNCTIONS ---
from plotting_helpers import *

# --- 5. SELECTION CUT FUNCTIONS ---
def apply_custom_cuts(reco_jets, config, key):
    """
    Apply custom cuts to a jet collection.

    Parameters
    ----------
    reco_jets : awkward.Array
        Jet collection (offline or L1).
    config : dict
        Global CONFIG dict.
    key : str
        Either "offline" or "l1" to select the appropriate config.
    """
    subcfg = config[key]

    pt_cut = subcfg["pt_cut"]
    eta_cut = subcfg["eta_cut"]
    b_tag_cut = subcfg["b_tag_cut"]
    tagger_name = subcfg["tagger_name"]

    print(f"\nApplying custom pT cut of {pt_cut} GeV for {key} jets...")
    pt_mask = reco_jets.pt > pt_cut
    eta_mask = abs(reco_jets.eta) < eta_cut
    final_mask = pt_mask & eta_mask

    print(f"Applying custom cuts for {tagger_name} ({key})...")

    if key == "offline":
        charm_veto_cut = subcfg["charm_veto_cut"]
        electron_veto_cut = subcfg["electron_veto_cut"]
        muon_veto_cut = subcfg["muon_veto_cut"]

        if tagger_name.startswith("btagPNet"):
            b_jet_mask = (reco_jets.btagPNetB > b_tag_cut)
            charm_veto_mask = (reco_jets.btagPNetCvB < charm_veto_cut)
            final_mask = final_mask & charm_veto_mask & b_jet_mask

        elif tagger_name.startswith("btagUParTAK4"):
            b_jet_mask = (reco_jets.btagUParTAK4probb > b_tag_cut)
            charm_veto_mask = (reco_jets.btagUParTAK4CvB < charm_veto_cut)
            electron_veto_mask = (reco_jets.btagUParTAK4Ele < electron_veto_cut)
            muon_veto_mask = (reco_jets.btagUParTAK4Mu < muon_veto_cut)
            final_mask = (
                final_mask
                & charm_veto_mask
                & electron_veto_mask
                & muon_veto_mask
                & b_jet_mask
            )

    elif key == "l1":
        # For L1, just apply the tagger cut generically
        tag_mask = getattr(reco_jets, tagger_name) > b_tag_cut
        final_mask = final_mask & tag_mask

    elif key == "l1ext":
        tag_mask = getattr(reco_jets, tagger_name) > b_tag_cut
        final_mask = final_mask & tag_mask

    reco_jets = reco_jets[final_mask]
    return reco_jets



In [ ]:
# for comparing the L1NG to the L1ExtJet, offline collections

import copy

with open("hh-bbbb-obj-config.json", "r") as config_file:
    CONFIG = json.load(config_file)

pt_bins = np.linspace(0, 500, 201)
eta_bins = np.linspace(-3, 3, 201)
b_tag_bins = np.linspace(0, 1, 101)

events = load_and_prepare_data(
    CONFIG["file_pattern"], 
    CONFIG["tree_name"], 
    [
        "GenPart", 
        CONFIG["offline"]["collection_name"], 
        CONFIG["l1"]["collection_name"],
        "L1puppiExtJetSC4"
    ], 
    CONFIG["max_events"]
)

upart_CONFIG = copy.deepcopy(CONFIG)
upart_CONFIG["offline"]["tagger_name"] = "btagUParTAK4B"

upart_events = load_and_prepare_data(
    upart_CONFIG["file_pattern"], 
    upart_CONFIG["tree_name"], 
    [
        upart_CONFIG["offline"]["collection_name"],
    ], 
    upart_CONFIG["max_events"],
    CONFIG=upart_CONFIG
)

upart_CONFIG = copy.deepcopy(CONFIG)
upart_CONFIG["offline"]["tagger_name"] = "btagUParTAK4B"

gen_b_quarks = select_gen_b_quarks_from_higgs(events)
gen_b_quarks = gen_b_quarks[(gen_b_quarks.pt > CONFIG["gen"]["pt_cut"]) & (abs(gen_b_quarks.eta) < CONFIG["gen"]["eta_cut"])]

reco_jets_offline = apply_custom_cuts(events[CONFIG["offline"]["collection_name"]], CONFIG, "offline")
reco_jets_offline_upart = apply_custom_cuts(upart_events[upart_CONFIG["offline"]["collection_name"]], upart_CONFIG, "offline")
reco_jets_l1ng = apply_custom_cuts(events[CONFIG["l1"]["collection_name"]], CONFIG, "l1")
reco_jets_l1ext = apply_custom_cuts(events[CONFIG["l1ext"]["collection_name"]], CONFIG, "l1ext")

# efficiency masks
b_quarks_is_matched_offline = get_efficiency_mask(gen_b_quarks, reco_jets_offline)
b_quarks_is_matched_offline_upart = get_efficiency_mask(gen_b_quarks, reco_jets_offline_upart)
b_quarks_is_matched_l1ng = get_efficiency_mask(gen_b_quarks, reco_jets_l1ng)
b_quarks_is_matched_l1ext = get_efficiency_mask(gen_b_quarks, reco_jets_l1ext)
eff_objects = [
    ("Offline", reco_jets_offline, b_quarks_is_matched_offline),
    ("Offline UParT", reco_jets_offline_upart, b_quarks_is_matched_offline_upart),
    ("L1NG", reco_jets_l1ng, b_quarks_is_matched_l1ng),
    ("L1ExtJet", reco_jets_l1ext, b_quarks_is_matched_l1ext)
]

# purity masks
is_reco_jet_pure_offline = get_purity_mask(gen_b_quarks, reco_jets_offline)
is_reco_jet_pure_offline_upart = get_purity_mask(gen_b_quarks, reco_jets_offline_upart)
is_reco_jet_pure_l1ng = get_purity_mask(gen_b_quarks, reco_jets_l1ng)
is_reco_jet_pure_l1ext = get_purity_mask(gen_b_quarks, reco_jets_l1ext)
purity_objects = [
    ("Offline", reco_jets_offline, is_reco_jet_pure_offline),
    ("Offline UParT", reco_jets_offline_upart, is_reco_jet_pure_offline_upart),
    ("L1NG", reco_jets_l1ng, is_reco_jet_pure_l1ng),
    ("L1ExtJet", reco_jets_l1ext, is_reco_jet_pure_l1ext)
]

print("Plotting reconstruction efficiencies for L1NG vs L1ExtJet...")
plot_kinematic_comparison(
    bins=np.linspace(0, 500, 51), variable="pt",
    xlabel=r"Generated b-quark $p_T$ [GeV]", title="Reco Efficiency vs. $p_T$",
    gen_particles=gen_b_quarks,
    objects=eff_objects
)
plot_kinematic_comparison(
    bins=np.linspace(-3, 3, 51), variable="eta",
    xlabel="Generated b-quark $\\eta$", title="Reco Efficiency vs. $\\eta$",
    gen_particles=gen_b_quarks,
    objects=eff_objects
)

plot_kinematic_comparison(
    bins=np.linspace(0, 500, 51), variable="pt",
    xlabel=r"Reconstructed b-quark $p_T$ [GeV]", title="Reco Purity vs. $p_T$",
    gen_particles=gen_b_quarks,
    objects=purity_objects,
    is_purity_plot=True
)
plot_kinematic_comparison(
    bins=np.linspace(-3, 3, 51), variable="eta",
    xlabel="Reconstructed b-quark $\\eta$", title="Reco Purity vs. $\\eta$",
    gen_particles=gen_b_quarks,
    objects=purity_objects,
    is_purity_plot=True
)


# plot_btag_map(reco_jets_l1ext, "btagScore", pt_bins, eta_bins)
# plot_btag_map(reco_jets_offline, CONFIG["offline"]["tagger_name"], pt_bins, eta_bins)
# plot_btag_map(reco_jets_offline_upart, "btagUParTAK4B", pt_bins, eta_bins)
# plot_btag_map(reco_jets_l1ng, CONFIG["l1"]["tagger_name"], pt_bins, eta_bins)

plt.hist(ak.flatten(getattr(reco_jets_l1ext, CONFIG["l1ext"]["tagger_name"])), bins=b_tag_bins, histtype="step", label="L1ExtJet B-Tag Score")
plt.hist(ak.flatten(getattr(reco_jets_l1ng, CONFIG["l1"]["tagger_name"])), bins=b_tag_bins, histtype="step", label="L1NG B-Tag Score")
plt.hist(ak.flatten(getattr(reco_jets_offline, CONFIG["offline"]["tagger_name"])), bins=b_tag_bins, histtype="step", label="Offline B-Tag Score")
plt.hist(ak.flatten(getattr(reco_jets_offline_upart, "btagUParTAK4B")), bins=b_tag_bins, histtype="step", label="Offline UParT B-Tag Score")
plt.legend()
plt.show()

# plot_signal_background_histogram(reco_jets_offline, is_reco_jet_pure_offline, pt_bins, "pt", r"Offline Jet $p_T$ [GeV]", "Offline Jet $p_T$ Distribution")
# plot_signal_background_histogram(reco_jets_offline_upart, is_reco_jet_pure_offline_upart, pt_bins, "pt", r"Offline UParT Jet $p_T$ [GeV]", "Offline UParT Jet $p_T$ Distribution")
# plot_signal_background_histogram(reco_jets_l1ng, is_reco_jet_pure_l1ng, pt_bins, "pt", r"L1NG Jet $p_T$ [GeV]", "L1NG Jet $p_T$ Distribution")
# plot_signal_background_histogram(reco_jets_l1ext, is_reco_jet_pure_l1ext, pt_bins, "pt", r"L1ExtJet Jet $p_T$ [GeV]", "L1ExtJet Jet $p_T$ Distribution")

# plot_signal_background_histogram(reco_jets_offline, is_reco_jet_pure_offline, np.linspace(-3, 3, 51), "eta", "Offline Jet $\\eta$", "Offline Jet $\\eta$ Distribution")
# plot_signal_background_histogram(reco_jets_offline_upart, is_reco_jet_pure_offline_upart, np.linspace(-3, 3, 51), "eta", "Offline UParT Jet $\\eta$", "Offline UParT Jet $\\eta$ Distribution")
# plot_signal_background_histogram(reco_jets_l1ng, is_reco_jet_pure_l1ng, np.linspace(-3, 3, 51), "eta", "L1NG Jet $\\eta$", "L1NG Jet $\\eta$ Distribution")
# plot_signal_background_histogram(reco_jets_l1ext, is_reco_jet_pure_l1ext, np.linspace(-3, 3, 51), "eta", "L1ExtJet Jet $\\eta$", "L1ExtJet Jet $\\eta$ Distribution")

# plot_signal_background_histogram(reco_jets_offline, is_reco_jet_pure_offline, np.linspace(0, 1, 51), CONFIG["offline"]["tagger_name"], "Offline Jet B-Tag Score", "Offline Jet B-Tag Score Distribution")
# plot_signal_background_histogram(reco_jets_offline_upart, is_reco_jet_pure_offline_upart, np.linspace(0, 1, 51), "btagUParTAK4B", "Offline UParT Jet B-Tag Score", "Offline UParT Jet B-Tag Score Distribution")
# plot_signal_background_histogram(reco_jets_l1ng, is_reco_jet_pure_l1ng, np.linspace(0, 1, 51), CONFIG["l1"]["tagger_name"], "L1NG Jet B-Tag Score", "L1NG Jet B-Tag Score Distribution")
# plot_signal_background_histogram(reco_jets_l1ext, is_reco_jet_pure_l1ext, np.linspace(0, 1, 51), CONFIG["l1ext"]["tagger_name"], "L1ExtJet Jet B-Tag Score", "L1ExtJet Jet B-Tag Score Distribution")

offline_roc = calculate_roc_points(
    reco_jets_offline, 
    is_reco_jet_pure_offline, 
    CONFIG["offline"]["tagger_name"]
)

offline_upart_roc = calculate_roc_points(
    reco_jets_offline_upart, 
    is_reco_jet_pure_offline_upart, 
    "btagUParTAK4B"
)

l1ng_roc = calculate_roc_points(
    reco_jets_l1ng, 
    is_reco_jet_pure_l1ng, 
    CONFIG["l1"]["tagger_name"]
)
l1ext_roc = calculate_roc_points(
    reco_jets_l1ext, 
    is_reco_jet_pure_l1ext, 
    CONFIG["l1ext"]["tagger_name"]
)

plot_roc_comparison([
    (f"Offline {CONFIG["offline"]["tagger_name"]}", offline_roc),
    (f"Offline UParT", offline_upart_roc),
    ("L1NG", l1ng_roc),
    ("L1ExtJet", l1ext_roc)
    ],
    working_point=0.001
)


mistag_offline, eff_offline, auc_offline, thresh_offline = offline_roc
mistag_offline_upart, eff_offline_upart, auc_offline_upart, thresh_offline_upart = offline_upart_roc
mistag_l1, eff_l1, auc_l1, thresh_l1 = l1ng_roc
mistag_l1ext, eff_l1ext, auc_l1ext, thresh_l1ext = l1ext_roc

def get_roc_point_at_efficiency(mistag, eff, thresh, target_eff):
    return [(m, e, th) for m, e, th in zip(mistag, eff, thresh) if e >= target_eff][-1]

def get_roc_point_at_mistag(mistag, eff, thresh, target_mistag):
    return [(m, e, th) for m, e, th in zip(mistag, eff, thresh) if m >= target_mistag][-1]

fpr_offline_tight, tpr_offline_tight, thresh_offline_tight = get_roc_point_at_mistag(mistag_offline, eff_offline, thresh_offline, 0.001)
fpr_offline_medium, tpr_offline_medium, thresh_offline_medium = get_roc_point_at_mistag(mistag_offline, eff_offline, thresh_offline, 0.01)
fpr_offline_loose, tpr_offline_loose, thresh_offline_loose = get_roc_point_at_mistag(mistag_offline, eff_offline, thresh_offline, 0.1)
print(f"\nOffline \nAUC: {auc_offline:.4f}")
print(f"Tight WP: TPR: {tpr_offline_tight * 100:.4f}%, 1/FPR: {1/fpr_offline_tight:.4f}, Threshold: {thresh_offline_tight:.4f}")
print(f"Medium WP: TPR: {tpr_offline_medium * 100:.4f}%, 1/FPR: {1/fpr_offline_medium:.4f}, Threshold: {thresh_offline_medium:.4f}")
print(f"Loose WP: TPR: {tpr_offline_loose * 100:.4f}%, 1/FPR: {1/fpr_offline_loose:.4f}, Threshold: {thresh_offline_loose:.4f}")

fpr_offline_upart_tight, tpr_offline_upart_tight, thresh_offline_upart_tight = get_roc_point_at_mistag(mistag_offline_upart, eff_offline_upart, thresh_offline_upart, 0.001)
fpr_offline_upart_medium, tpr_offline_upart_medium, thresh_offline_upart_medium = get_roc_point_at_mistag(mistag_offline_upart, eff_offline_upart, thresh_offline_upart, 0.01)
fpr_offline_upart_loose, tpr_offline_upart_loose, thresh_offline_upart_loose = get_roc_point_at_mistag(mistag_offline_upart, eff_offline_upart, thresh_offline_upart, 0.1)
print(f"\nOffline UParT \nAUC: {auc_offline_upart:.4f}")
print(f"Tight WP: TPR: {tpr_offline_upart_tight * 100:.4f}%, 1/FPR: {1/fpr_offline_upart_tight:.4f}, Threshold: {thresh_offline_upart_tight:.4f}")
print(f"Medium WP: TPR: {tpr_offline_upart_medium * 100:.4f}%, 1/FPR: {1/fpr_offline_upart_medium:.4f}, Threshold: {thresh_offline_upart_medium:.4f}")
print(f"Loose WP: TPR: {tpr_offline_upart_loose * 100:.4f}%, 1/FPR: {1/fpr_offline_upart_loose:.4f}, Threshold: {thresh_offline_upart_loose:.4f}")


fpr_l1_tight, tpr_l1_tight, thresh_l1_tight = get_roc_point_at_mistag(mistag_l1, eff_l1, thresh_l1, 0.001)
fpr_l1_medium, tpr_l1_medium, thresh_l1_medium = get_roc_point_at_mistag(mistag_l1, eff_l1, thresh_l1, 0.01)
fpr_l1_loose, tpr_l1_loose, thresh_l1_loose = get_roc_point_at_mistag(mistag_l1, eff_l1, thresh_l1, 0.1)
print(f"\nL1 \nAUC: {auc_l1:.4f}")
print(f"Tight WP: TPR: {tpr_l1_tight * 100:.4f}%, 1/FPR: {1/fpr_l1_tight:.4f}, Threshold: {thresh_l1_tight:.4f}")
print(f"Medium WP: TPR: {tpr_l1_medium * 100:.4f}%, 1/FPR: {1/fpr_l1_medium:.4f}, Threshold: {thresh_l1_medium:.4f}")
print(f"Loose WP: TPR: {tpr_l1_loose * 100:.4f}%, 1/FPR: {1/fpr_l1_loose:.4f}, Threshold: {thresh_l1_loose:.4f}")

fpr_l1ext_tight, tpr_l1ext_tight, thresh_l1ext_tight = get_roc_point_at_mistag(mistag_l1ext, eff_l1ext, thresh_l1ext, 0.001)
fpr_l1ext_medium, tpr_l1ext_medium, thresh_l1ext_medium = get_roc_point_at_mistag(mistag_l1ext, eff_l1ext, thresh_l1ext, 0.01)
fpr_l1ext_loose, tpr_l1ext_loose, thresh_l1ext_loose = get_roc_point_at_mistag(mistag_l1ext, eff_l1ext, thresh_l1ext, 0.1)
print(f"\nL1ExtJet \nAUC: {auc_l1ext:.4f}")
print(f"Tight WP: TPR: {tpr_l1ext_tight * 100:.4f}%, 1/FPR: {1/fpr_l1ext_tight:.4f}, Threshold: {thresh_l1ext_tight:.4f}")
print(f"Medium WP: TPR: {tpr_l1ext_medium * 100:.4f}%, 1/FPR: {1/fpr_l1ext_medium:.4f}, Threshold: {thresh_l1ext_medium:.4f}")
print(f"Loose WP: TPR: {tpr_l1ext_loose * 100:.4f}%, 1/FPR: {1/fpr_l1ext_loose:.4f}, Threshold: {thresh_l1ext_loose:.4f}")

In [ ]:
# enquire about the hwPt plot -> solved
# Answer: hwPT: raw pT from FPGAs -> cotinuous values in actual pt after a potentilly non-linear mapping from the raw integer values to a continuous pt space - can be ignored for now.
reco_jets_l1 = events[CONFIG["l1"]["collection_name"]]
plt.hist(ak.flatten(reco_jets_l1.hwPt * reco_jets_l1.ptCorrection), bins=np.linspace(0, 1000, 101), histtype="step", label="hwPt")
plt.hist(ak.flatten(reco_jets_l1.pt * reco_jets_l1.ptCorrection), bins=np.linspace(0, 1000, 101), histtype="step", label="pt")
plt.legend()
plt.show()


# plt.hist(ak.flatten(reco_jets_l1.c_v_b_score), bins=np.linspace(0, 1, 101), histtype="step", label="CvB Score")
plt.hist(ak.flatten(reco_jets_l1.cTagScore), bins=np.linspace(0, 1, 101), histtype="step", label="C Score")
plt.hist(ak.flatten(reco_jets_l1.bTagScore), bins=np.linspace(0, 1, 101), histtype="step", label="B Score")
# plt.hist(ak.flatten(reco_jets_l1.b_v_udscg_score), bins=np.linspace(0, 1, 101), histtype="step", label="BvUDSCG Score")
plt.hist(ak.flatten(reco_jets_l1.udsTagScore), bins=np.linspace(0, 1, 101), histtype="step", label="UDS Score")
# plt.hist(ak.flatten(reco_jets_l1.eTagScore), bins=np.linspace(0, 1, 101), histtype="step", label="E Score")
plt.legend()
plt.show()


In [ ]:
# to see and plot the horns
plot_attr_vs_var_proj(events, CONFIG["gen"]["collection_name"], "pt", "eta", 
    bins_attr=np.linspace(0, 500, 101), bins_var=np.linspace(-3, 3, 101),
    xlabel="Jet $\\eta$", ylabel="Jet $p_T$ [GeV]",
    title="Gen Jet $p_T$ vs. $\\eta$"
)
plot_attr_vs_var_proj(events, CONFIG["offline"]["collection_name"], "pt", "eta", 
    bins_attr=np.linspace(0, 500, 201), bins_var=np.linspace(-4, 4, 201),
    xlabel="Jet $\\eta$", ylabel="Jet $p_T$ [GeV]",
    title="Offline Jet $p_T$ vs. $\\eta$"
)
plot_attr_vs_var_proj(events, CONFIG["offline"]["collection_name"], "pt", "eta", 
    bins_attr=np.linspace(0, 500, 201), bins_var=np.linspace(-4, 4, 201),
    xlabel="Jet $\\eta$", ylabel="Jet $p_T$ [GeV]",
    title="Offline Pure Jet $p_T$ vs. $\\eta$",
    mask=is_reco_jet_pure_offline
)
plot_attr_vs_var_proj(events, CONFIG["offline"]["collection_name"], "pt", "eta", 
    bins_attr=np.linspace(0, 500, 201), bins_var=np.linspace(-4, 4, 201),
    xlabel="Jet $\\eta$", ylabel="Jet $p_T$ [GeV]",
    title="Offline Impure Jet $p_T$ vs. $\\eta$",
    mask=~is_reco_jet_pure_offline
)
# plot_attr_vs_var_proj(events, CONFIG["l1"]["collection_name"], "pt", "eta", 
#     bins_attr=np.linspace(0, 500, 101), bins_var=np.linspace(-3, 3, 101),
#     xlabel="Jet $\\eta$", ylabel="Jet $p_T$ [GeV]",
#     title="L1NG Jet $p_T$ vs. $\\eta$"
# )
# plot_attr_vs_var_proj(events, CONFIG["l1ext"]["collection_name"], "pt", "eta",
#     bins_attr=np.linspace(0, 500, 101), bins_var=np.linspace(-3, 3, 101),
#     xlabel="Jet $\\eta$", ylabel="Jet $p_T$ [GeV]",
#     title="L1ext Jet $p_T$ vs. $\\eta$"
# )

In [ ]:
# avg b_tag scores
plot_objs_pt = [
    (reco_jets_offline, is_reco_jet_pure_offline,
    CONFIG["offline"]["tagger_name"], "pt", np.linspace(0, 500, 51),
    "Jet $p_T$ [GeV]", "B-Tag Score",
    "Offline Jet B-Tag Score vs. $p_T$"),
    (reco_jets_offline_upart, is_reco_jet_pure_offline_upart,
    "btagUParTAK4B", "pt", np.linspace(0, 500, 51),
    "Jet $p_T$ [GeV]", "B-Tag Score",
    "Offline UParT Jet B-Tag Score vs. $p_T$"),
    (reco_jets_l1ng, is_reco_jet_pure_l1ng,
    CONFIG["l1"]["tagger_name"], "pt", np.linspace(0, 500, 51),
    "Jet $p_T$ [GeV]", "B-Tag Score",
    "L1NG Jet B-Tag Score vs. $p_T$"),
    (reco_jets_l1ext, is_reco_jet_pure_l1ext,
    CONFIG["l1ext"]["tagger_name"], "pt", np.linspace(0, 500, 51),
    "Jet $p_T$ [GeV]", "B-Tag Score",
    "L1ExtJet Jet B-Tag Score vs. $p_T$")
]

for obj in plot_objs_pt:
    plot_avg_attr_vs_var(*obj)


plot_objs_eta = [
    (reco_jets_offline, is_reco_jet_pure_offline,
    CONFIG["offline"]["tagger_name"], "eta", np.linspace(-3, 3, 51),
    "Jet $\\eta$", "B-Tag Score",
    "Offline Jet B-Tag Score vs. $\\eta$"),
    (reco_jets_offline_upart, is_reco_jet_pure_offline_upart,
    "btagUParTAK4B", "eta", np.linspace(-3, 3, 51),
    "Jet $\\eta$", "B-Tag Score",
    "Offline UParT Jet B-Tag Score vs. $\\eta$"),
    (reco_jets_l1ng, is_reco_jet_pure_l1ng,
    CONFIG["l1"]["tagger_name"], "eta", np.linspace(-3, 3, 51),
    "Jet $\\eta$", "B-Tag Score",
    "L1NG Jet B-Tag Score vs. $\\eta$"),
    (reco_jets_l1ext, is_reco_jet_pure_l1ext,
    CONFIG["l1ext"]["tagger_name"], "eta", np.linspace(-3, 3, 51),
    "Jet $\\eta$", "B-Tag Score",
    "L1ExtJet Jet B-Tag Score vs. $\\eta$")
]
for obj in plot_objs_eta:
    plot_avg_attr_vs_var(*obj)


# reco_offline_hun_mask = get_purity_mask_hungarian(gen_b_quarks, reco_jets_offline)
# plot_kinematic_comparison(
#     bins=np.linspace(0, 500, 51), variable="pt",
#     xlabel=r"Generated b-quark $p_T$ [GeV]", title="Reco Purity vs. $p_T$ (Hungarian Matching)",
#     gen_particles=gen_b_quarks,
#     objects=[
#         ("Offline Jet (Hungarian Matching)", reco_jets_offline, reco_offline_hun_mask),
#         ("Offline Jet", reco_jets_offline, is_reco_jet_pure_offline)
#     ],
#     is_purity_plot=True
# )

In [ ]:
# all histograms together for offline jets

plt.hist(ak.flatten((gen_b_quarks.eta)), bins=np.linspace(-4, 4, 81), histtype="step", label="Gen b-quarks")
plt.hist(ak.flatten((reco_jets_offline.eta)), bins=np.linspace(-4, 4, 81), histtype="step", label="Reco Offline Jets")
plt.hist(ak.flatten((reco_jets_offline[is_reco_jet_pure_offline].eta)), bins=np.linspace(-4, 4, 81), histtype="step", label="Reco signal Offline Jets")
plt.hist(ak.flatten((gen_b_quarks[b_quarks_is_matched_offline].eta)), bins=np.linspace(-4, 4, 81), histtype="step", label="Matched Gen Jets")
plt.hist(ak.flatten((gen_b_quarks[~b_quarks_is_matched_offline].eta)), bins=np.linspace(-4, 4, 81), histtype="step", label="Unmatched Gen Jets")
# plt.hist(ak.flatten((reco_jets_offline[~is_reco_jet_pure_offline].eta)), bins=np.linspace(-4, 4, 51), histtype="step", label="Reco background Offline Jets")
plt.legend()
plt.xlabel("Eta")
plt.ylabel("Counts")
plt.show()

plt.hist(ak.flatten((gen_b_quarks.pt)), bins=np.linspace(0, 500, 101), histtype="step", label="Gen b-quarks")
plt.hist(ak.flatten((reco_jets_offline.pt)), bins=np.linspace(0, 500, 101), histtype="step", label="Reco Offline Jets")
plt.hist(ak.flatten((reco_jets_offline[is_reco_jet_pure_offline].pt)), bins=np.linspace(0, 500, 101), histtype="step", label="Reco signal Offline Jets")
plt.hist(ak.flatten((gen_b_quarks[b_quarks_is_matched_offline].pt)), bins=np.linspace(0, 500, 101), histtype="step", label="Matched Gen Jets")
plt.hist(ak.flatten((gen_b_quarks[~b_quarks_is_matched_offline].pt)), bins=np.linspace(0, 500, 101), histtype="step", label="Unmatched Gen Jets")
plt.legend()
plt.xlabel("pt")
plt.ylabel("Counts")
plt.show()

plt.hist(ak.flatten((gen_b_quarks.eta)), bins=np.linspace(-4, 4, 51), histtype="step", label="Gen b-quarks")
plt.hist(ak.flatten((reco_jets_offline.eta)), bins=np.linspace(-4, 4, 51), histtype="step", label="Reco Offline Jets")
plt.hist(ak.flatten((reco_jets_offline[is_reco_jet_pure_offline].eta)), bins=np.linspace(-4, 4, 51), histtype="step", label="Reco signal Offline Jets")
plt.hist(ak.flatten((reco_jets_offline[~is_reco_jet_pure_offline].eta)), bins=np.linspace(-4, 4, 51), histtype="step", label="Reco background Offline Jets")
plt.legend()
plt.xlabel("Eta")
plt.ylabel("Counts")
plt.show()

plt.hist(ak.flatten((gen_b_quarks.pt)), bins=np.linspace(0, 500, 101), histtype="step", label="Gen b-quarks")
plt.hist(ak.flatten((reco_jets_offline.pt)), bins=np.linspace(0, 500, 101), histtype="step", label="Reco Offline Jets")
plt.hist(ak.flatten((reco_jets_offline[is_reco_jet_pure_offline].pt)), bins=np.linspace(0, 500, 101), histtype="step", label="Reco signal Offline Jets")
plt.hist(ak.flatten((reco_jets_offline[~is_reco_jet_pure_offline].pt)), bins=np.linspace(0, 500, 101), histtype="step", label="Reco background Offline Jets")
plt.legend()
plt.xlabel("Pt")
plt.ylabel("Counts")
plt.xlim(150, 400)
plt.ylim(0, 2000)
plt.show()


plt.hist(ak.flatten(reco_jets_offline.PNetRegPtRawRes), bins=np.linspace(-1, 1, 101), histtype="step", label="Offline PNetRegPtRawRes")
plt.show()

plot_matching_criteria(gen_b_quarks, reco_jets_offline)
plot_matching_criteria(gen_b_quarks, reco_jets_offline_upart)
plot_matching_criteria(gen_b_quarks, reco_jets_l1ng)
plot_matching_criteria(gen_b_quarks, reco_jets_l1ext)

In [ ]:
# Resolution and Scale Plots
# TODO: look at Prijith's work on jet energy scale and resolution
def calculate_jet_resolutions(gen_particles, reco_objects, CONFIG=None):
    """
    Calculates pT and Energy resolution using vectorized 
    Cross-Mutual Nearest Neighbor matching (1-to-1).
    
    Returns:
        gen_pt_flat:  The pT of the matched Gen particles (for the x-axis)
        pt_res_flat:  (Reco - Gen) / Gen pT resolution
        e_res_flat:   (Reco - Gen) / Gen Energy resolution
    """
    if CONFIG is None:
        with open("hh-bbbb-obj-config.json", "r") as config_file:
            CONFIG = json.load(config_file)
            
    # 1. Prepare 4-vectors for broadcasting
    # gen shape: (events, n_gen, 1)
    # reco shape: (events, 1, n_reco)
    gen_vec = gen_particles.vector[:, :, None]
    reco_vec = reco_objects.vector[:, None, :]
    
    # 2. Calculate DeltaR Matrix (All-to-All)
    delta_r_matrix = gen_vec.deltaR(reco_vec)
    
    # 3. Find Cross-Mutual Nearest Neighbors (Vectorized 1-to-1)
    
    # For every Gen particle, find index of closest Reco
    # shape: (events, n_gen)
    idx_closest_reco_to_gen = ak.argmin(delta_r_matrix, axis=2)
    
    # For every Reco jet, find index of closest Gen
    # shape: (events, n_reco)
    idx_closest_gen_to_reco = ak.argmin(delta_r_matrix, axis=1)
    
    # 4. Check the Cross-Match Condition
    # We look at the Gen particles. We ask: 
    # "Is the Reco jet closest to ME (Gen) also pointing back at ME?"
    
    # Get the index of the Gen particle that the closest Reco jet points to
    # We use the advanced indexing: array[indices]
    back_check_idx = idx_closest_gen_to_reco[idx_closest_reco_to_gen]
    
    # Create indices for comparison (0, 1, 2...)
    gen_indices = ak.local_index(gen_particles, axis=1)
    
    # A match is valid if:
    # 1. The indices match (Mutual agreement)
    # 2. The distance is within the cone size
    
    # Get the actual dR values for the closest matches
    min_dr_values = ak.min(delta_r_matrix, axis=2)
    
    is_mutual_match = (
        (back_check_idx == gen_indices) & 
        (min_dr_values < CONFIG["matching_cone_size"])
    )
    
    # 5. Extract the Matched Objects
    matched_gen = gen_particles[is_mutual_match]
    
    # We need to pull the specific Reco jets that matched these Gen particles
    # idx_closest_reco_to_gen contains the indices of the Reco jets we want
    # We apply the boolean mask to the INDICES first
    matched_reco_indices = idx_closest_reco_to_gen[is_mutual_match]
    
    # Now select those jets from the reco collection
    matched_reco = reco_objects[matched_reco_indices]
    
    # 6. Calculate Resolutions
    # Formula: (Reco - Gen) / Gen
    
    # Pt Resolution
    pt_res = (matched_reco.vector.pt - matched_gen.vector.pt) / matched_gen.vector.pt
    
    # Energy Resolution
    matched_reco.vector["energy"] = np.sqrt((matched_reco.vector.pt * np.cosh(matched_reco.vector.eta)) ** 2 + (matched_reco.vector.mass) ** 2)
    e_res = (matched_reco.vector.energy - matched_gen.vector.energy) / matched_gen.vector.energy
    
    # Flatten arrays for easy plotting
    # We return the Gen Pt as the x-axis variable
    return (
        ak.to_numpy(ak.flatten(matched_gen.vector.eta)),
        ak.to_numpy(ak.flatten(matched_gen.vector.pt)), 
        ak.to_numpy(ak.flatten(pt_res)),
        ak.to_numpy(ak.flatten(e_res))
    )

def plot_resolution_vs_var(objects):
    """
    Bins the data by Gen pT and calculates the Mean and Width (StdDev) 
    of the resolution in each bin.
    """
    plt.figure(figsize=(10, 8))
    for gen_var, resolution_values, bins, y_label, x_label, title in objects:
        bin_centers = 0.5 * (bins[1:] + bins[:-1])
        
        means = []
        widths = []
        errors = [] # Error on the width calculation
        
        # Digitize: Find which bin each event belongs to
        # indices 1 to len(bins)-1 are valid bins
        bin_indices = np.digitize(gen_var, bins)
        
        for i in range(1, len(bins)):
            # Select data for this specific bin
            data_in_bin = resolution_values[bin_indices == i]
            
            if len(data_in_bin) > 10: # Require minimum stats
                # Calculate Mean (Bias)
                mu = np.mean(data_in_bin)
                
                # Calculate Width (Resolution)
                # Standard Deviation is simple, but IQR/2 is more robust against tails
                sigma = np.std(data_in_bin) 
                
                # Error on std dev estimate approx: sigma / sqrt(2N)
                err = sigma / np.sqrt(2 * len(data_in_bin))
                
                means.append(mu)
                widths.append(sigma)
                errors.append(err)
            else:
                means.append(np.nan)
                widths.append(np.nan)
                errors.append(0)

        # --- Plotting ---
        # Top Panel: Resolution (Width)
        plt.subplot(2, 1, 1)
        plt.errorbar(bin_centers, widths, yerr=errors, fmt='o-', capsize=5, label=f'{y_label} Resolution ($\sigma$)')
        plt.ylabel(f"Resolution")
        plt.title(title)
        plt.grid(True, linestyle='--', alpha=0.6)
        plt.legend()
        
        # Bottom Panel: Scale/Bias (Mean)
        plt.subplot(2, 1, 2)
        plt.errorbar(bin_centers, means, fmt='s--', capsize=5, label=f'{y_label} Scale (Mean)')
        plt.axhline(0, color='black', linestyle='-', linewidth=1)
        plt.xlabel(x_label)
        plt.ylabel("Scale (Bias)")
        plt.grid(True, linestyle='--', alpha=0.6)
        plt.legend()
    
    plt.tight_layout()
    plt.show()

flat_gen_eta, flat_gen_pt, flat_pt_res, flat_e_res = calculate_jet_resolutions(gen_b_quarks, reco_jets_offline)
flat_gen_eta_l1ng, flat_gen_pt_l1ng, flat_pt_res_l1ng, flat_e_res_l1ng = calculate_jet_resolutions(gen_b_quarks, reco_jets_l1ng)
flat_gen_eta_l1ext, flat_gen_pt_l1ext, flat_pt_res_l1ext, flat_e_res_l1ext = calculate_jet_resolutions(gen_b_quarks, reco_jets_l1ext)

pt_res_objects = [
    (flat_gen_pt, flat_pt_res, 
     np.linspace(0, 500, 51),
     "Offline Jet $p_T$",
     "Generated $p_T$ [GeV]",
     "Jet $p_T$ Resolution and Scale vs. Generated $p_T$"),
     (flat_gen_pt_l1ng, flat_pt_res_l1ng, 
     np.linspace(0, 500, 51),
     "L1NG Jet $p_T$",
     "Generated $p_T$ [GeV]",
     "Jet $p_T$ Resolution and Scale vs. Generated $p_T$"),
     (flat_gen_pt_l1ext, flat_pt_res_l1ext, 
     np.linspace(0, 500, 51),
     "L1Ext Jet $p_T$",
     "Generated $p_T$ [GeV]",
     "Jet $p_T$ Resolution and Scale vs. Generated $p_T$")
]
plot_resolution_vs_var(
    pt_res_objects
)

eta_res_objects = [
    (flat_gen_eta, flat_e_res, 
     np.linspace(-3, 3, 51),
     "PNet Jet $p_T$",
     "Generated $\\eta$",
     "Jet $p_T$ Resolution and Scale vs. Generated $\\eta$"),

     (flat_gen_eta_l1ng, flat_e_res_l1ng, 
     np.linspace(-3, 3, 51),
     "L1NG Jet $p_T$",
     "Generated $\\eta$",
     "Jet $p_T$ Resolution and Scale vs. Generated $\\eta$"),

     (flat_gen_eta_l1ext, flat_e_res_l1ext, 
     np.linspace(-3, 3, 51),
     "L1Ext Jet $p_T$",
     "Generated $\\eta$",
     "Jet $p_T$ Resolution and Scale vs. Generated $\\eta$")
]
plot_resolution_vs_var(
    eta_res_objects
)

plot_avg_attr_vs_var(
    reco_jets_offline, is_reco_jet_pure_offline,
    "PNetRegPtRawRes", "pt", np.linspace(0, 500, 51),
    "Jet $p_T$ [GeV]", "PNetRegPtRawRes",
    "Offline Jet PNetRegPtRawRes vs. $p_T$"
)
plot_avg_attr_vs_var(
    reco_jets_offline, is_reco_jet_pure_offline,
    "PNetRegPtRawRes", "eta", np.linspace(-3, 3, 51),
    "Jet $\\eta$", "PNetRegPtRawRes",
    "Offline Jet PNetRegPtRawRes vs. $\\eta$"
)
plot_avg_attr_vs_var(
    reco_jets_offline_upart, is_reco_jet_pure_offline,
    "UParTAK4RegPtRawRes", "pt", np.linspace(0, 500, 51),
    "Jet $p_T$ [GeV]", "UParTAK4RegPtRawRes",
    "Offline UParT Jet UParTAK4RegPtRawRes vs. $p_T$"
)
plot_avg_attr_vs_var(
    reco_jets_offline_upart, is_reco_jet_pure_offline,
    "UParTAK4RegPtRawRes", "eta", np.linspace(-3, 3, 51),
    "Jet $\\eta$", "UParTAK4RegPtRawRes",
    "Offline UParT Jet UParTAK4RegPtRawRes vs. $\\eta$"
)

In [ ]:
# Storing hungarian masks to disk to speed up future analysis

import copy

events = load_and_prepare_data(
    CONFIG["file_pattern"], 
    CONFIG["tree_name"], 
    [
        "GenPart", 
        CONFIG["offline"]["collection_name"], 
        CONFIG["l1"]["collection_name"],
        "L1puppiExtJetSC4"
    ], 
    CONFIG["max_events"]
)

upart_CONFIG = copy.deepcopy(CONFIG)
upart_CONFIG["offline"]["tagger_name"] = "btagUParTAK4B"

upart_events = load_and_prepare_data(
    upart_CONFIG["file_pattern"], 
    upart_CONFIG["tree_name"], 
    [
        upart_CONFIG["offline"]["collection_name"],
    ], 
    upart_CONFIG["max_events"],
    CONFIG=upart_CONFIG
)

upart_CONFIG = copy.deepcopy(CONFIG)
upart_CONFIG["offline"]["tagger_name"] = "btagUParTAK4B"

gen_b_quarks = select_gen_b_quarks_from_higgs(events)
gen_b_quarks = gen_b_quarks[(gen_b_quarks.pt > CONFIG["gen"]["pt_cut"]) & (abs(gen_b_quarks.eta) < CONFIG["gen"]["eta_cut"])]

reco_jets_offline = events[CONFIG["offline"]["collection_name"]]
reco_jets_l1ng = events[CONFIG["l1"]["collection_name"]]
reco_jets_l1ext = events["L1puppiExtJetSC4"]
reco_jets_offline_upart = upart_events[upart_CONFIG["offline"]["collection_name"]]


print("Generating efficiency masks")
b_quarks_is_matched_offline = get_efficiency_mask_hungarian(gen_b_quarks, reco_jets_offline)
b_quarks_is_matched_offline_upart = get_efficiency_mask_hungarian(gen_b_quarks, reco_jets_offline_upart)
b_quarks_is_matched_l1ng = get_efficiency_mask_hungarian(gen_b_quarks, reco_jets_l1ng)
b_quarks_is_matched_l1ext = get_efficiency_mask_hungarian(gen_b_quarks, reco_jets_l1ext)

print("Generating purity masks")
is_reco_jet_pure_offline = get_purity_mask_hungarian(gen_b_quarks, reco_jets_offline)
is_reco_jet_pure_offline_upart = get_purity_mask_hungarian(gen_b_quarks, reco_jets_offline_upart)
is_reco_jet_pure_l1 = get_purity_mask_hungarian(gen_b_quarks, reco_jets_l1ng)
is_reco_jet_pure_l1ext = get_purity_mask_hungarian(gen_b_quarks, reco_jets_l1ext)


# print("Storing masks to disk")
mask_list = [
    b_quarks_is_matched_offline,
    b_quarks_is_matched_offline_upart,
    b_quarks_is_matched_l1ng,
    b_quarks_is_matched_l1ext,
    is_reco_jet_pure_offline,
    is_reco_jet_pure_offline_upart,
    is_reco_jet_pure_l1,
    is_reco_jet_pure_l1ext
]
key_list = [
    "b_quarks_is_matched_offline",
    "b_quarks_is_matched_offline_upart",
    "b_quarks_is_matched_l1ng",
    "b_quarks_is_matched_l1ext",
    "is_reco_jet_pure_offline",
    "is_reco_jet_pure_offline_upart",
    "is_reco_jet_pure_l1",
    "is_reco_jet_pure_l1ext"
]
with uproot.recreate(f"mask_data.root") as file:
    for mask, key in zip(mask_list, key_list):
        print(f"Storing mask with {ak.sum(mask)} true entries out of {len(ak.flatten(mask))} total entries ({ak.sum(mask)/len(ak.flatten(mask))*100:.2f}%)")
        file[key] = mask

# Reloading
print("Reloading masks from disk and verifying integrity")
for mask, key in zip(mask_list, key_list):
    with uproot.open("mask_data.root") as file:
        # .array() converts it back to an Awkward Array
        loaded_mask = file[key].array()
    assert ak.all(loaded_mask == mask), f"Loaded mask {key} does not match the original mask!"

In [ ]:
mask, key = mask_list[0], key_list[0]
with uproot.open("mask_data.root") as file:
    # .array() converts it back to an Awkward Array
    loaded_mask = file[key]

file[b_quarks_is_matched_offline]

In [ ]:
reco_mass_offline = reco_jets_offline.et
mass_mat_offline = reco_mass_offline[:, :, None] + reco_mass_offline[:, None, :]
# mass_mat_offline = mass_mat_offline - 120
mass_mat_offline

In [ ]:
def gen_b_masks(events):
    """
    For each event, checks if at least n b-jets are found within the top k jets.
    Returns a boolean mask per event.
    """

    gen_b_from_higgs = select_gen_b_quarks_from_higgs(events)

    pt_cut_gen = CONFIG["gen"]["pt_cut"]
    eta_cut_gen = CONFIG["gen"]["eta_cut"]

    gen_b_from_higgs = gen_b_from_higgs[(gen_b_from_higgs.pt > pt_cut_gen) & (abs(gen_b_from_higgs.eta) < eta_cut_gen)]

    base_jets_offline = events[CONFIG["offline"]["collection_name"]]
    base_l1_jets = events[CONFIG["l1"]["collection_name"]]

    pt_ordered_offline = base_jets_offline[ak.argsort(base_jets_offline.vector.pt, ascending=False)]
    pt_ordered_l1 = base_l1_jets[ak.argsort(base_l1_jets.vector.pt, ascending=False)]

    b_score_ordered_offline = base_jets_offline[ak.argsort(getattr(base_jets_offline, CONFIG["offline"]["tagger_name"]), ascending=False)]
    b_score_ordered_l1 = base_l1_jets[ak.argsort(getattr(base_l1_jets, CONFIG["l1"]["tagger_name"]), ascending=False)]

    true_pt_offline_mask = get_purity_mask(gen_b_from_higgs, pt_ordered_offline)
    true_pt_l1_mask = get_purity_mask(gen_b_from_higgs, pt_ordered_l1)

    true_btag_offline_mask = get_purity_mask(gen_b_from_higgs, b_score_ordered_offline)
    true_btag_l1_mask = get_purity_mask(gen_b_from_higgs, b_score_ordered_l1)

    pt_ordered_offline = pt_ordered_offline[true_pt_offline_mask]
    pt_ordered_l1 = pt_ordered_l1[true_pt_l1_mask]

    b_score_ordered_offline = b_score_ordered_offline[true_btag_offline_mask]
    b_score_ordered_l1 = b_score_ordered_l1[true_btag_l1_mask]

    return {"gen_b_from_higgs": gen_b_from_higgs,
            "ordered": (pt_ordered_offline, pt_ordered_l1, b_score_ordered_offline, b_score_ordered_l1), 
            "masks": (true_pt_offline_mask, true_pt_l1_mask, true_btag_offline_mask, true_btag_l1_mask)
            }

def find_n_jets_rolling(gen_b_from_higgs, ordered, masks, n, k):
    top_k_pt_offline = ordered[0][:, :k]
    top_k_pt_l1 = ordered[1][:, :k]

    top_k_btag_offline = ordered[2][:, :k]
    top_k_btag_l1 = ordered[3][:, :k]

    eff_pt_offline = ak.sum(ak.num(top_k_pt_offline)) / ak.sum(ak.num(gen_b_from_higgs))
    eff_pt_l1 = ak.sum(ak.num(top_k_pt_l1)) / ak.sum(ak.num(gen_b_from_higgs))

    eff_btag_offline = ak.sum(ak.num(top_k_btag_offline)) / ak.sum(ak.num(gen_b_from_higgs))
    eff_btag_l1 = ak.sum(ak.num(top_k_btag_l1)) / ak.sum(ak.num(gen_b_from_higgs))

    true_pt_offline_mask = masks[0][:, :k]
    more_than_n_mask_offline = ak.sum(true_pt_offline_mask, axis=1) >= n
    more_than_n_efficiency_offline = ak.sum(more_than_n_mask_offline) / len(ak.flatten(gen_b_from_higgs))

    true_pt_l1_mask = masks[1][:, :k]
    more_than_n_mask_l1 = ak.sum(true_pt_l1_mask, axis=1) >= n
    more_than_n_efficiency_l1 = ak.sum(more_than_n_mask_l1) / len(ak.flatten(gen_b_from_higgs))

    true_btag_offline_mask = masks[2][:, :k]
    more_than_n_b_mask_offline = ak.sum(true_btag_offline_mask, axis=1) >= n
    more_than_n_b_efficiency_offline = ak.sum(more_than_n_b_mask_offline) / len(ak.flatten(gen_b_from_higgs))

    true_btag_l1_mask = masks[3][:, :k]
    more_than_n_b_mask_l1 = ak.sum(true_btag_l1_mask, axis=1) >= n
    more_than_n_b_efficiency_l1 = ak.sum(more_than_n_b_mask_l1) / len(ak.flatten(gen_b_from_higgs))

    return {"more_than_n_pt": (more_than_n_efficiency_offline, more_than_n_efficiency_l1),
            "more_than_n_b": (more_than_n_b_efficiency_offline, more_than_n_b_efficiency_l1),
            "pt_eff": (eff_pt_offline, eff_pt_l1),
            "btag_eff": (eff_btag_offline, eff_btag_l1)
           }


more_than_n_eff_pt_offline = []
more_than_n_eff_pt_l1 = []

more_than_n_eff_btag_offline = []
more_than_n_eff_btag_l1 = []

ordered_and_masks = gen_b_masks(events)
gen_b_from_higgs = ordered_and_masks["gen_b_from_higgs"]
ordered = ordered_and_masks["ordered"]
masks = ordered_and_masks["masks"]

n = 2

for k in range(20):
    out_dict = find_n_jets_rolling(gen_b_from_higgs, ordered, masks, n, k+1)

    more_than_n_efficiency_offline, more_than_n_efficiency_l1 = out_dict["more_than_n_pt"]
    more_than_n_eff_pt_offline.append(more_than_n_efficiency_offline)
    more_than_n_eff_pt_l1.append(more_than_n_efficiency_l1)

    more_than_n_efficiency_offline, more_than_n_efficiency_l1 = out_dict["more_than_n_b"]
    more_than_n_eff_btag_offline.append(more_than_n_efficiency_offline)
    more_than_n_eff_btag_l1.append(more_than_n_efficiency_l1)

print(f"\nPlotting Top-N Jet Efficiencies for N = {n}...")
plt.figure(figsize=(10, 6))
plt.step(range(20), more_than_n_eff_pt_offline, where='mid', label='Offline pT')
plt.step(range(20), more_than_n_eff_pt_l1, where='mid', label='L1 pT')
plt.xlabel("k (Number of Top Jets Considered)")
plt.ylabel(f"Efficiency of Finding at least {n} b-jets")
plt.title(f"Efficiency of Finding at least {n} b-jets vs. Top k Jets Considered")
plt.grid(True, linestyle='--', alpha=0.6)
plt.legend()
plt.show()

plt.figure(figsize=(10, 6))
plt.step(range(20), more_than_n_eff_btag_offline, where='mid', label='Offline BTag')
plt.step(range(20), more_than_n_eff_btag_l1, where='mid', label='L1 BTag')
plt.xlabel("k (Number of Top Jets Considered)")
plt.ylabel(f"Efficiency of Finding at least {n} b-jets")
plt.title(f"Efficiency of Finding at least {n} b-jets vs. Top k Jets Considered")
plt.grid(True, linestyle='--', alpha=0.6)
plt.legend()
plt.show()

In [ ]:
# TODO: fix this optmisation code

from scipy.optimize import minimize_scalar
import warnings

def calculate_efficiency_for_cut(reco_jets, gen_b_quarks, tagger_name, cut_value):
    """
    Calculates the b-tagging efficiency for a given cut on the tagger.
    Efficiency = (Number of true b-jets passing the cut) / (Total number of true b-jets)
    """
    # Find which reco jets are actually b-jets
    is_pure_mask = get_purity_mask(gen_b_quarks, reco_jets)
    true_b_jets = reco_jets[is_pure_mask]
    
    # Find how many of those true b-jets pass the cut
    passing_jets = true_b_jets[getattr(true_b_jets, tagger_name) > cut_value]
    
    # Calculate efficiency
    n_passing = ak.sum(ak.num(passing_jets, axis=0))
    n_total = ak.sum(ak.num(true_b_jets, axis=0))
    
    if n_total == 0:
        return 0.0
        
    return n_passing / n_total

def objective_function(cut_value, reco_jets, gen_b_quarks, tagger_name, target_efficiency):
    """
    Objective function for the optimizer.
    Calculates the squared difference between the current and target efficiency.
    """
    current_efficiency = calculate_efficiency_for_cut(reco_jets, gen_b_quarks, tagger_name, cut_value)
    return (current_efficiency - target_efficiency)**2

def find_cut_for_efficiency(reco_jets, gen_b_quarks, tagger_name, target_efficiency):
    """
    Uses scipy.optimize.minimize_scalar to find the b-tag cut value that
    results in the target b-tagging efficiency.
    """
    print(f"Optimizing cut for '{tagger_name}' to achieve {target_efficiency:.1%} efficiency...")
    
    # The optimizer needs a function that takes only the variable to be optimized (cut_value)
    # We use a lambda function to "freeze" the other arguments.
    obj_func = lambda cut: objective_function(cut, reco_jets, gen_b_quarks, tagger_name, target_efficiency)
    
    # We use minimize_scalar to find the minimum of our objective function.
    # This is equivalent to finding where the efficiency is closest to our target.
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", RuntimeWarning)
        result = minimize_scalar(
            obj_func,
            bounds=(0.0, 1.0),  # B-tag scores are between 0 and 1
            method='bounded'
        )
    
    optimal_cut = result.x
    final_efficiency = calculate_efficiency_for_cut(reco_jets, gen_b_quarks, tagger_name, optimal_cut)
    
    print(f"Optimization complete:")
    print(f"  - Target Efficiency: {target_efficiency:.3f}")
    print(f"  - Optimal Cut Value: {optimal_cut:.4f}")
    print(f"  - Resulting Efficiency: {final_efficiency:.3f}")
    print("-" * 30)
    
    return optimal_cut, final_efficiency

# --- Run the Optimization ---

# Ensure we have the necessary data loaded
if 'events' not in locals():
    print("Reloading data as 'events' was not found in the environment.")
    with open("hh-bbbb-obj-config.json", "r") as config_file:
        CONFIG = json.load(config_file)
    events = run_analysis(CONFIG)

# Select gen b-quarks with fiducial cuts
gen_b_quarks_for_opt = select_gen_b_quarks_from_higgs(events)
gen_b_quarks_for_opt = gen_b_quarks_for_opt[
    (gen_b_quarks_for_opt.pt > CONFIG["gen"]["pt_cut"]) & 
    (abs(gen_b_quarks_for_opt.eta) < CONFIG["gen"]["eta_cut"])
]

# Get the reconstructed jet collections
reco_jets_offline_for_opt = events[CONFIG["offline"]["collection_name"]]
reco_jets_l1_for_opt = events[CONFIG["l1"]["collection_name"]]

# --- Define Target Efficiencies and Run ---
target_eff_offline = 0.70  # e.g., 70%
target_eff_l1 = 0.70       # e.g., 50%

# Find the optimal cut for the Offline jets
optimal_cut_offline, final_eff_offline = find_cut_for_efficiency(
    reco_jets_offline_for_opt,
    gen_b_quarks_for_opt,
    CONFIG["offline"]["tagger_name"],
    target_eff_offline
)

# Find the optimal cut for the L1 jets
optimal_cut_l1, final_eff_l1 = find_cut_for_efficiency(
    reco_jets_l1_for_opt,
    gen_b_quarks_for_opt,
    CONFIG["l1"]["tagger_name"],
    target_eff_l1
)
